# Example 2: Writing Data
First we import the necessary modules

In [1]:
from pythermondt.readers import LocalReader
from pythermondt.writers import LocalWriter, S3Writer

Now reader and writer objects can be created.

**Note** In this example the S3Writer class is used to upload the data to an AWS S3 bucket. For this to work you need setup your AWS credentials. This can be done in several ways (see [here](https://boto3.amazonaws.com/v1/documentation/api/latest/guide/configuration.html#guide-configuration))

In [2]:
reader = LocalReader(pattern="./example_data/example-data*.hdf5")
writer = LocalWriter(destination_folder="./example_data")
uploader = S3Writer(bucket="ffg-bp", destination_folder="example2_writing_data")

Now we can load data using the reader object, manipulate it, and write it back to a file using the writer object. Writers always write their data to an HDF5 File. The data can be read back using a reader object.

The files can also be viewed using: [HDF5 Viewer](https://myhdf5.hdfgroup.org/)

In [3]:
for i, container in enumerate(reader):
    container.add_attributes("/MetaData", author="Max Mustermann", version=12.6, owners=["max", "moritz"], ids={"max": 1, "moritz": 2}) 
    writer.write(container, f"example-write{i}.hdf5")

Additionally, the Data can also be uploaded to an AWS S3 Bucket using the AWSWriter object.

In [4]:
for i, container in enumerate(reader):
    container.add_attributes("/MetaData", author="Max Mustermann", version=12.6, owners=["max", "moritz"], ids={"max": 1, "moritz": 2}) 
    uploader.write(container, f"example-write{i}.hdf5")

Uploading file: example-write0.hdf5:   0%|          | 0.00/3.34M [00:00<?, ?B/s]

Uploading file: example-write1.hdf5:   0%|          | 0.00/3.08M [00:00<?, ?B/s]

Uploading file: example-write2.hdf5:   0%|          | 0.00/3.18M [00:00<?, ?B/s]